In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 2.1MB 11.8MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 901kB 54.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=c08e3e4490dc94fd69e877f1fc94698375351e65e601aa04871531fddd17c711
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [ ]:
!pip install underthesea

     |████████████████████████████████| 7.5MB 8.6MB/s 
     |████████████████████████████████| 245kB 39.4MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 1.3MB 37.2MB/s 
     |████████████████████████████████| 747kB 21.2MB/s 
     |████████████████████████████████| 1.1MB 33.0MB/s 
     |████████████████████████████████| 2.9MB 37.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=043e3b36009bf7efc32ff6bbe9972120c4191c87f57f1ebb39c57e83f0ed8929
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.5.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.1 which is incompatible.
ERROR: sentence-transformers 1.0.4 has requirement t

In [ ]:
from sentence_transformers import util
import torch

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [ ]:
from underthesea import sent_tokenize

In [ ]:
import json

In [ ]:
def find_sentence(corpus, start_answer, context):
  '''
  Find sentence contains answer
  * corpus: list sentences of context
  * start_answer: the begin position of answer in context
  '''
  start, end = 0, -1
  for sentence in corpus:
    start = end+1
    end = start+len(sentence)
    if start<=start_answer<end: break
  idx_answer_in_sent = start_answer - context.find(sentence)
  return sentence, idx_answer_in_sent

In [ ]:
def update_answer_index(sentence, new_context, idx_answer_in_sent):
  '''
  Update new answer_start
  * sentence: sentence contains answer
  * new_context: new context
  * idx_answer_in_sent: the begin position of answer in sentence 
  '''
  i = new_context.find(sentence)
  answer_start_new = i + idx_answer_in_sent
  return answer_start_new

In [ ]:
def find_top_k(top_k, model, question, corpus, corpus_embeddings):
  if len(corpus)>top_k:
        query_embedding = model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()

        top_results = torch.topk(cos_scores, k=top_k)


        sentence_new_context = []
        for score, idx in zip(top_results[0], top_results[1]):
          sentence_new_context.append(corpus[idx])
        
        return sentence_new_context
  else:
        return corpus

#### Train set

In [ ]:
def update_data_train(top_k, data):
  new_data_train = {'version': "1.1", 'data': []}
  for id_a, a in enumerate(data):

    new_data_train['data'].append({"title": a['title'], "paragraphs": []})

    for paragraph in a['paragraphs']:

      context = paragraph['context']
      corpus = sent_tokenize(context)
      corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

      for qa in paragraph['qas']:
        question = qa['question']
        answer = qa['answers'][0]['text']
        start_answer = qa['answers'][0]['answer_start']

        sentence_answer, idx_answer_in_sent = find_sentence(corpus, start_answer, context)
      
        if len(corpus)>top_k:

          sentence_new_context = find_top_k(top_k, model, question, corpus, corpus_embeddings)
          
          if sentence_answer in sentence_new_context:

            new_context = ''
            for pa in corpus:
              if pa in sentence_new_context: new_context+= pa+' '
            new_context = new_context[:-1]
            if answer in new_context:
    
              answer_start_new = update_answer_index(sentence_answer, new_context, idx_answer_in_sent)

              #print(new_context)
              #print(question)
              #print(answer)
              #print(str(start_answer)+' -->  '+str(answer_start_new))
              #print(new_context[answer_start_new:answer_start_new+len(answer)])
              #print('----------------------------------')

              answers = [{'answer_start': answer_start_new, 'text': answer}]
              qas_ = [{'question': question, 'answers': answers, 'id': qa['id']}]
              new_data_train['data'][id_a]["paragraphs"].append({'context':new_context, 'qas':qas_   })

        else:
        
          answers = [{'answer_start': start_answer, 'text': answer}]
          qas_ = [{'question': question, 'answers': answers, 'id': qa['id']}]
          new_data_train['data'][id_a]["paragraphs"].append({'context':context, 'qas':qas_   })
  return new_data_train

In [ ]:
with open("./ViQuAD1.1/train_ViQuAD.json") as f:
  data =  json.load(f)['data']

In [ ]:
new_data_train = update_data_train(5, data)

In [ ]:
with open("./ViQuAD1.1/top_5/train_ViQuAD_top_5.json", "w") as write_file:
  json.dump(new_data_train, write_file)

#### Dev and test sets

In [ ]:
def update_data_dev(top_k, data):
  new_data = {'version': "1.1", 'data': []}
  for id_a, a in enumerate(data):

    new_data['data'].append({"title": a['title'], "paragraphs": []})

    for paragraph in a['paragraphs']:

      context = paragraph['context']
      corpus = sent_tokenize(context)
      corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

      for qa in paragraph['qas']:
        question = qa['question']

        answers = qa['answers']
      
        if len(corpus)>top_k:

          sentence_new_context = find_top_k(top_k, model, question, corpus,corpus_embeddings)

          new_context = ''
          for pa in corpus:
            if pa in sentence_new_context: new_context+= pa+' '
          new_context = new_context[:-1]
    
          #answer_start_new = update_answer_index(sentence_answer, new_context, idx_answer_in_sent)

          #print(new_context)
          #print(question)
          #print(answer)
          #print(str(start_answer)+' -->  '+str(answer_start_new))
          #print(new_context[answer_start_new:answer_start_new+len(answer)])
          #print('----------------------------------')


          qas_ = [{'question': question, 'answers': answers, 'id': qa['id']}]
          new_data['data'][id_a]["paragraphs"].append({'context':new_context, 'qas':qas_   })

        else:
        
          qas_ = [{'question': question, 'answers': answers, 'id': qa['id']}]
          new_data['data'][id_a]["paragraphs"].append({'context':context, 'qas':qas_   })
  return new_data

In [ ]:
with open("./ViQuAD1.1/dev_ViQuAD.json") as f:
  data =  json.load(f)['data']

In [ ]:
new_data_dev = update_data_dev(5, data)

In [ ]:
with open("./ViQuAD1.1/top_5/dev_ViQuAD_top_5.json","w") as f:
  json.dump(new_data_dev, f)

In [ ]:
with open("./ViQuAD1.1/test_ViQuAD.json") as f:
  data =  json.load(f)['data']

In [ ]:
new_data_test = update_data_dev(5, data)

In [ ]:
with open("./ViQuAD1.1/top_5/test_ViQuAD_top_5.json","w") as f:
  json.dump(new_data_test, f)